# Genie Top200 Chart

In [1]:
import requests
import pandas as pd

### 1. 인터넷 상에서 데이터 가져오기

In [2]:
# Genie Top200
url = 'https://www.genie.co.kr/chart/top200'
req = requests.get(url)
html = req.text
html

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [3]:
# Chrome User-Agent
header = {'User-Agent':
          'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}
req = requests.get(url, headers=header)
html = req.text
#html

In [4]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

### 2. 찾으려고 하는 데이터의 태그 찾기

In [5]:
# <table class="list-wrap">
table = soup.select_one('table.list-wrap')
trs = table.select('tr.list')
len(trs)

50

In [6]:
trs = soup.select('tr.list')
len(trs)

50

### 3. 여러개의 데이터중 하나를 선택해서 원하는 정보 추출

In [7]:
# rank, title, artist, album 찾기
tr = trs[0]

In [8]:
# rank
tr.select_one('.number').get_text()

'1\n                                        \n                                    \n유지\n\n'

In [9]:
tr.select_one('.number').get_text().split('\n')[0]

'1'

In [10]:
rank = int(tr.select_one('.number').get_text().split('\n')[0])
rank

1

In [11]:
# title
tr.select_one('.info').select_one('.title').get_text()

'\n                                        \n                                            \n                                        \n                                        \n                                        \n                                        \n                                            \n                                                사진을 보다가'

In [12]:
title = tr.select_one('.info').select_one('.title').get_text().strip()
title

'사진을 보다가'

In [13]:
# artist
tr.select_one('.info').select_one('.artist').get_text()

'다비치'

In [14]:
artist = tr.select_one('.info').select_one('.artist').get_text().strip()
artist

'다비치'

In [15]:
# album
tr.select_one('.info').select_one('.albumtitle').get_text()

'사진을 보다가 (REVIBE Vol.2)'

In [16]:
album = tr.select_one('.info').select_one('.albumtitle').get_text().strip()
album

'사진을 보다가 (REVIBE Vol.2)'

### 4. 한 페이지에 있는 모든 데이터를 반복문으로 가져오기

In [17]:
rank_list, title_list, artist_list, album_list = [],[],[],[]
for tr in trs:
    rank = int(tr.select_one('.number').get_text().split('\n')[0])
    title = tr.select_one('.info').select_one('.title').get_text().strip()
    artist = tr.select_one('.info').select_one('.artist').get_text().strip()
    album = tr.select_one('.info').select_one('.albumtitle').get_text().strip()
    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)

In [18]:
df = pd.DataFrame({
    '순위':rank_list, '곡명':title_list,
    '가수':artist_list, '앨범':album_list
})
df.tail()

,순위,곡명,가수,앨범
45,46,At My Worst,Pink Sweat$,The Prelude
46,47,가을 우체국 앞에서,김대명,슬기로운 의사생활 시즌2 OST Part 2 (tvN 목요드라마)
47,48,에잇 (Prod. & Feat. SUGA of BTS),아이유 (IU),에잇
48,49,사랑하니까 (2021),이보람 (씨야),사랑하니까 (2021)
49,50,추적이는 여름 비가 되어,장범준,장범준 싱글


In [19]:
lines = []
for tr in trs:
    rank = int(tr.select_one('.number').get_text().split('\n')[0])
    title = tr.select_one('.info').select_one('.title').get_text().strip()
    artist = tr.select_one('.info').select_one('.artist').get_text().strip()
    album = tr.select_one('.info').select_one('.albumtitle').get_text().strip()
    lines.append([rank, title, artist, album])

In [20]:
df = pd.DataFrame(lines, columns=['순위','곡명','가수','앨범'])
df.head()

,순위,곡명,가수,앨범
0,1,사진을 보다가,다비치,사진을 보다가 (REVIBE Vol.2)
1,2,신호등,이무진,신호등
2,3,Queendom,Red Velvet (레드벨벳),Queendom - The 6th Mini Album
3,4,낙하 (With 아이유),AKMU (악뮤),NEXT EPISODE
4,5,바라만 본다,MSG워너비 (M.O.M),MSG워너비 1집


### 5. 모든 페이지 데이터 가져오기

In [21]:
sub = 'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20210817&hh=14&rtm=Y&pg='
rank_list, title_list, artist_list, album_list = [],[],[],[]
for page in range(1,5):
    url = f'{sub}{page}'
    req = requests.get(url, headers=header)
    soup = BeautifulSoup(req.text, 'html.parser')
    trs = soup.select('tr.list')
    for tr in trs:
        rank = int(tr.select_one('.number').get_text().split('\n')[0])
        title = tr.select_one('.info').select_one('.title').get_text().strip()
        artist = tr.select_one('.info').select_one('.artist').get_text().strip()
        album = tr.select_one('.info').select_one('.albumtitle').get_text().strip()
        rank_list.append(rank)
        title_list.append(title)
        artist_list.append(artist)
        album_list.append(album)

In [22]:
df = pd.DataFrame({
    '순위':rank_list, '곡명':title_list,
    '가수':artist_list, '앨범':album_list
})
df.tail()

,순위,곡명,가수,앨범
195,196,아무도 잠들지 마라 (Nessun Dorma),김호중,The Classic Album
196,197,All I Wanna Do (K) (Feat. Hoody & Loco),박재범,EVERYTHING YOU WANTED
197,198,해야 해,2PM,MUST
198,199,"너에게 난, 나에게 넌",미도와 파라솔,슬기로운 의사생활 OST Part 12 (tvN 목요드라마)
199,200,파도,폴킴,첫번째 수학여행 'Summer : re'


In [23]:
df.to_csv('GenieTop200(210817).csv', index=False)